In [3]:
# Life expectancy data source: OECD (https://www.oecd.org/en/data/indicators/life-expectancy-at-birth.html).
# requires pandas, so before executing run `pip install pandas`

In [4]:
import sys; sys.path.append("../")
import os
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "lifetime_in_weeks.settings")
django.setup()

from datetime import date
import pandas as pd
pd.set_option("display.max_columns", 50)
from lifetime.models import LifetimeExpectancy

In [5]:
# download from https://www.oecd.org/en/data/indicators/life-expectancy-at-birth.html
# or https://data-explorer.oecd.org/vis?lc=en&ac=false&tm=DF_LE&pg=0&snb=1&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_HEALTH_STAT%40DF_LE&df[ag]=OECD.ELS.HD&df[vs]=&pd=%2C&dq=.A...Y0........&to[TIME_PERIOD]=false
oecd = pd.read_csv("oecd_life_expectancy.csv")
oecd.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,MEASURE,Measure,UNIT_MEASURE,Unit of measure,AGE,Age,SEX,Sex,SOCIO_ECON_STATUS,Socio-economic status,DEATH_CAUSE,Cause of death,CALC_METHODOLOGY,Calculation methodology,GESTATION_THRESHOLD,Gestation period threshold,HEALTH_STATUS,Health status,DISEASE,Disease,CANCER_SITE,Cancer site,TIME_PERIOD,Time period,OBS_VALUE,Observation value,DECIMALS,Decimals,OBS_STATUS,Observation status,OBS_STATUS2,Observation status 2,OBS_STATUS3,Observation status 3,UNIT_MULT,Unit multiplier
0,DATAFLOW,OECD.ELS.HD:DSD_HEALTH_STAT@DF_LE(1.0),Life expectancy,I,AUS,Australia,A,Annual,LFEXPDMF,Life expectancy difference (male-female),Y,Years,Y60,60 years,_T,Total,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,2020,NaN,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DATAFLOW,OECD.ELS.HD:DSD_HEALTH_STAT@DF_LE(1.0),Life expectancy,I,AUS,Australia,A,Annual,LFEXPDMF,Life expectancy difference (male-female),Y,Years,Y60,60 years,_T,Total,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,2019,NaN,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DATAFLOW,OECD.ELS.HD:DSD_HEALTH_STAT@DF_LE(1.0),Life expectancy,I,AUS,Australia,A,Annual,LFEXP,Life expectancy,Y,Years,Y60,60 years,F,Female,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,1963,NaN,19.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DATAFLOW,OECD.ELS.HD:DSD_HEALTH_STAT@DF_LE(1.0),Life expectancy,I,AUS,Australia,A,Annual,LFEXP,Life expectancy,Y,Years,Y60,60 years,F,Female,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,1962,NaN,19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DATAFLOW,OECD.ELS.HD:DSD_HEALTH_STAT@DF_LE(1.0),Life expectancy,I,AUS,Australia,A,Annual,LFEXP,Life expectancy,Y,Years,Y60,60 years,F,Female,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,_Z,Not applicable,1961,NaN,19.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# download from https://www.iso.org/iso-3166-country-codes.html
# or https://www.iso.org/obp/ui/#search
iso_codes = pd.read_csv("iso_codes.csv")
iso_codes.head()

,English short name,French short name,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,Afghanistan (l'),AF,AFG,4
1,Albania,Albanie (l'),AL,ALB,8
2,Algeria,Algérie (l'),DZ,DZA,12
3,American Samoa,Samoa américaines (les),AS,ASM,16
4,Andorra,Andorre (l'),AD,AND,20


In [7]:
oecd = oecd[oecd["Measure"] == "Life expectancy"]
oecd = oecd[oecd["AGE"] == "Y0"]
oecd = oecd[["REF_AREA", "SEX", "TIME_PERIOD", "OBS_VALUE"]]
oecd.head()

,REF_AREA,SEX,TIME_PERIOD,OBS_VALUE
431,AUS,_T,2018,82.7
432,AUS,_T,2017,82.5
433,AUS,_T,2016,82.4
434,AUS,_T,2015,82.4
435,AUS,_T,2014,82.3


In [8]:
oecd_columns = oecd.columns.tolist()
oecd = oecd.merge(iso_codes, left_on="REF_AREA", right_on="Alpha-3 code")
oecd = oecd[oecd_columns + ["Alpha-2 code"]]
oecd.head()

,REF_AREA,SEX,TIME_PERIOD,OBS_VALUE,Alpha-2 code
0,AUS,_T,2018,82.7,AU
1,AUS,_T,2017,82.5,AU
2,AUS,_T,2016,82.4,AU
3,AUS,_T,2015,82.4,AU
4,AUS,_T,2014,82.3,AU


In [9]:
oecd = oecd.rename(columns={
    "Alpha-2 code": "country_code",
    "SEX": "sex",
    "TIME_PERIOD": "year",
    "OBS_VALUE": "life_expectancy"
})
oecd = oecd.drop(columns=["REF_AREA"])
oecd = oecd[["country_code", "sex", "year", "life_expectancy"]]
oecd.head()

,country_code,sex,year,life_expectancy
0,AU,_T,2018,82.7
1,AU,_T,2017,82.5
2,AU,_T,2016,82.4
3,AU,_T,2015,82.4
4,AU,_T,2014,82.3


In [10]:
countries = oecd["country_code"].unique().tolist() + [None]
sexes = oecd["sex"].unique()
years = range(1960, 2024 + 1)
index = pd.MultiIndex.from_product(
        [countries, sexes, years],
        names=["country_code", "sex", "year"]
    )
index

MultiIndex([('AU', '_T', 1960),
            ('AU', '_T', 1961),
            ('AU', '_T', 1962),
            ('AU', '_T', 1963),
            ('AU', '_T', 1964),
            ('AU', '_T', 1965),
            ('AU', '_T', 1966),
            ('AU', '_T', 1967),
            ('AU', '_T', 1968),
            ('AU', '_T', 1969),
            ...
            ( nan,  'M', 2015),
            ( nan,  'M', 2016),
            ( nan,  'M', 2017),
            ( nan,  'M', 2018),
            ( nan,  'M', 2019),
            ( nan,  'M', 2020),
            ( nan,  'M', 2021),
            ( nan,  'M', 2022),
            ( nan,  'M', 2023),
            ( nan,  'M', 2024)],
           names=['country_code', 'sex', 'year'], length=9750)

In [11]:
# Reindex the original dataframe
oecd = oecd.set_index(['country_code', "sex", 'year']).reindex(index)
# Reset index to get country and year back as columns
oecd = oecd.reset_index()
# Group by country and sex and forward fill, then backward fill values
oecd['life_expectancy'] = (oecd.groupby(['country_code', "sex"])['life_expectancy']).apply(lambda x: x.ffill().bfill()).reset_index().set_index('level_2')["life_expectancy"]
oecd.head()

,country_code,sex,year,life_expectancy
0,AU,_T,1960,71.0
1,AU,_T,1961,71.0
2,AU,_T,1962,71.0
3,AU,_T,1963,71.0
4,AU,_T,1964,71.0


In [12]:
oecd[oecd["country_code"].isna()]

,country_code,sex,year,life_expectancy
9555,NaN,_T,1960,NaN
9556,NaN,_T,1961,NaN
9557,NaN,_T,1962,NaN
9558,NaN,_T,1963,NaN
9559,NaN,_T,1964,NaN
...,...,...,...,...
9745,NaN,M,2020,NaN
9746,NaN,M,2021,NaN
9747,NaN,M,2022,NaN
9748,NaN,M,2023,NaN


In [13]:
# download from https://ourworldindata.org/grapher/life-expectancy?time=1960..latest&country=~OWID_WRL
world = pd.read_csv("world_life_expectancy.csv")
world.head()

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0
0,World,OWID_WRL,1960,47.6972
1,World,OWID_WRL,1961,50.3587
2,World,OWID_WRL,1962,53.1245
3,World,OWID_WRL,1963,53.5761
4,World,OWID_WRL,1964,54.1638


In [14]:
# download from https://ourworldindata.org/grapher/life-expectation-at-birth-by-sex?time=1960..latest
world_by_sex = pd.read_csv("wolrd_life_expectancy_by_sex.csv")
world_by_sex.head()

,Entity,Code,Year,Period life expectancy - Sex: female - Age: 0,Period life expectancy - Sex: male - Age: 0
0,World,OWID_WRL,1960,49.4175,46.0358
1,World,OWID_WRL,1961,52.1529,48.6136
2,World,OWID_WRL,1962,54.9706,51.3166
3,World,OWID_WRL,1963,55.4417,51.7468
4,World,OWID_WRL,1964,56.0605,52.3000


In [15]:
world = world.merge(world_by_sex, on="Year", suffixes=("", "_by_sex"))
world

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,Entity_by_sex,Code_by_sex,Period life expectancy - Sex: female - Age: 0,Period life expectancy - Sex: male - Age: 0
0,World,OWID_WRL,1960,47.6972,World,OWID_WRL,49.4175,46.0358
1,World,OWID_WRL,1961,50.3587,World,OWID_WRL,52.1529,48.6136
2,World,OWID_WRL,1962,53.1245,World,OWID_WRL,54.9706,51.3166
3,World,OWID_WRL,1963,53.5761,World,OWID_WRL,55.4417,51.7468
4,World,OWID_WRL,1964,54.1638,World,OWID_WRL,56.0605,52.3000
...,...,...,...,...,...,...,...,...
57,World,OWID_WRL,2017,72.3267,World,OWID_WRL,74.9747,69.7603
58,World,OWID_WRL,2018,72.5759,World,OWID_WRL,75.1996,70.0279
59,World,OWID_WRL,2019,72.7897,World,OWID_WRL,75.4191,70.2365
60,World,OWID_WRL,2020,72.0361,World,OWID_WRL,74.7891,69.4084


In [16]:
last_year = world[world["Year"] == world["Year"].max()].iloc[0]
this_year = date.today().year
for year in range(last_year["Year"] + 1, this_year + 1):
    row = last_year.copy()
    row["Year"] = year
    world.loc[world.index.max() + 1] = row
world.tail()

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,Entity_by_sex,Code_by_sex,Period life expectancy - Sex: female - Age: 0,Period life expectancy - Sex: male - Age: 0
60,World,OWID_WRL,2020,72.0361,World,OWID_WRL,74.7891,69.4084
61,World,OWID_WRL,2021,71.0479,World,OWID_WRL,73.8070,68.4372
62,World,OWID_WRL,2022,71.0479,World,OWID_WRL,73.8070,68.4372
63,World,OWID_WRL,2023,71.0479,World,OWID_WRL,73.8070,68.4372
64,World,OWID_WRL,2024,71.0479,World,OWID_WRL,73.8070,68.4372


In [17]:
world = world.rename(columns={
    "Year": "year",
    "Period life expectancy at birth - Sex: all - Age: 0": "_T",
    "Period life expectancy - Sex: female - Age: 0": "F",
    "Period life expectancy - Sex: male - Age: 0": "M"
})
world = world[["year", "_T", "F", "M"]]
world.head()

,year,_T,F,M
0,1960,47.6972,49.4175,46.0358
1,1961,50.3587,52.1529,48.6136
2,1962,53.1245,54.9706,51.3166
3,1963,53.5761,55.4417,51.7468
4,1964,54.1638,56.0605,52.3000


In [18]:
_T = world[["year", "_T"]]
_T["country_code"] = None
_T["sex"] = "_T"
_T = _T.rename(columns={"_T": "life_expectancy"})
_T = _T[["country_code", "sex", "year", "life_expectancy"]]
_T

/tmp/ipykernel_6083/3696856019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _T["country_code"] = None


,country_code,sex,year,life_expectancy
0,None,_T,1960,47.6972
1,None,_T,1961,50.3587
2,None,_T,1962,53.1245
3,None,_T,1963,53.5761
4,None,_T,1964,54.1638
...,...,...,...,...
60,None,_T,2020,72.0361
61,None,_T,2021,71.0479
62,None,_T,2022,71.0479
63,None,_T,2023,71.0479


In [19]:
M = world[["year", "M"]]
M["country_code"] = None
M["sex"] = "M"
M = M.rename(columns={"M": "life_expectancy"})
M = M[["country_code", "sex", "year", "life_expectancy"]]
M

/tmp/ipykernel_6083/2793680316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M["country_code"] = None


,country_code,sex,year,life_expectancy
0,None,M,1960,46.0358
1,None,M,1961,48.6136
2,None,M,1962,51.3166
3,None,M,1963,51.7468
4,None,M,1964,52.3000
...,...,...,...,...
60,None,M,2020,69.4084
61,None,M,2021,68.4372
62,None,M,2022,68.4372
63,None,M,2023,68.4372


In [20]:
F = world[["year", "F"]]
F["country_code"] = None
F["sex"] = "F"
F = F.rename(columns={"F": "life_expectancy"})
F = F[["country_code", "sex", "year", "life_expectancy"]]
F

/tmp/ipykernel_6083/971492245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F["country_code"] = None


,country_code,sex,year,life_expectancy
0,None,F,1960,49.4175
1,None,F,1961,52.1529
2,None,F,1962,54.9706
3,None,F,1963,55.4417
4,None,F,1964,56.0605
...,...,...,...,...
60,None,F,2020,74.7891
61,None,F,2021,73.8070
62,None,F,2022,73.8070
63,None,F,2023,73.8070


In [21]:
oecd = pd.concat([oecd, _T, M, F])
oecd["sex"] = oecd["sex"].replace("_T", "O")
oecd

,country_code,sex,year,life_expectancy
0,AU,O,1960,71.0000
1,AU,O,1961,71.0000
2,AU,O,1962,71.0000
3,AU,O,1963,71.0000
4,AU,O,1964,71.0000
...,...,...,...,...
60,None,F,2020,74.7891
61,None,F,2021,73.8070
62,None,F,2022,73.8070
63,None,F,2023,73.8070


In [22]:
oecd_dicts = oecd.to_dict(orient="records")

In [23]:
objects_to_create = [
    LifetimeExpectancy(
        country=d["country_code"],
        sex=d["sex"],
        birth_year=d["year"],
        life_expectancy=d["life_expectancy"]
    ) for d in oecd_dicts 
]
objects_to_create[:5]

[<LifetimeExpectancy: LifetimeExpectancy object (None)>,
 <LifetimeExpectancy: LifetimeExpectancy object (None)>,
 <LifetimeExpectancy: LifetimeExpectancy object (None)>,
 <LifetimeExpectancy: LifetimeExpectancy object (None)>,
 <LifetimeExpectancy: LifetimeExpectancy object (None)>]

In [24]:
await LifetimeExpectancy.objects.abulk_create(objects_to_create)

[<LifetimeExpectancy: LifetimeExpectancy object (1)>,
 <LifetimeExpectancy: LifetimeExpectancy object (2)>,
 <LifetimeExpectancy: LifetimeExpectancy object (3)>,
 <LifetimeExpectancy: LifetimeExpectancy object (4)>,
 <LifetimeExpectancy: LifetimeExpectancy object (5)>,
 <LifetimeExpectancy: LifetimeExpectancy object (6)>,
 <LifetimeExpectancy: LifetimeExpectancy object (7)>,
 <LifetimeExpectancy: LifetimeExpectancy object (8)>,
 <LifetimeExpectancy: LifetimeExpectancy object (9)>,
 <LifetimeExpectancy: LifetimeExpectancy object (10)>,
 <LifetimeExpectancy: LifetimeExpectancy object (11)>,
 <LifetimeExpectancy: LifetimeExpectancy object (12)>,
 <LifetimeExpectancy: LifetimeExpectancy object (13)>,
 <LifetimeExpectancy: LifetimeExpectancy object (14)>,
 <LifetimeExpectancy: LifetimeExpectancy object (15)>,
 <LifetimeExpectancy: LifetimeExpectancy object (16)>,
 <LifetimeExpectancy: LifetimeExpectancy object (17)>,
 <LifetimeExpectancy: LifetimeExpectancy object (18)>,
 <LifetimeExpectanc

In [26]:
last_obj = await LifetimeExpectancy.objects.alast()
last_obj.sex

'F'